## Calc AV function
```
Option Explicit
Function CalcAV(Optional vTIERNUM As String) As String
    
    process_plan_data
    SetStatusMsg (vTIERNUM)
    
    SetContTable_Combined
    SetContTable_Med_and_Rx
    
    If gCB_INT_DEDUCT_IND = 1 Then
    
        CombinedMacro_v2
        CombinedMacro2_v2
        
    ElseIf gCB_INT_DEDUCT_IND = 0 And gCB_SEP_MOOP_IND = 0 Then
    
        SepDeductCombinedMOOP_v2
        SepDeductCombinedMOOP2_v2
        
    ElseIf gCB_INT_DEDUCT_IND = 0 And gCB_SEP_MOOP_IND = 1 Then
        
        SepMacro_v2
        SepMacro2_v2

    End If
       
    CalcAV = gAV

End Function
Function SetStatusMsg(Optional vTIERNUM As String)
    
    Dim n As Integer
    Dim p As Integer
    Dim zeroDeduct_med As Integer
    Dim zeroDeduct_rx As Integer
    
    zeroDeduct_med = 0
    zeroDeduct_rx = 0
    
    If gCB_INT_DEDUCT_IND = 1 Then
        If AVCalcTab.Range("nrDEDUCT" & vTIERNUM).value = 0 Then
            zeroDeduct_med = 1
            zeroDeduct_rx = 1
        End If
    Else
        If AVCalcTab.Range("nrDEDUCT_MED" & vTIERNUM).value = 0 Then zeroDeduct_med = 1
        If AVCalcTab.Range("nrDEDUCT_RX" & vTIERNUM).value = 0 Then zeroDeduct_rx = 1
    End If
    
    If zeroDeduct_med = 0 Then
        For n = LBound(gSTDED_ARR) To UBound(gSTDED_ARR)
            If gSTDED_ARR(n) = 0 And IsEmpty(AVCalcTab.Range(gMedSvcCopayNR_Arr(n) & vTIERNUM)) = True Then
                gNoteMsg.XSTD = 1
            End If
        Next
    End If

    If zeroDeduct_rx = 0 Then
        For p = LBound(gSTDEDRX_ARR) To UBound(gSTDEDRX_ARR)
            If gSTDEDRX_ARR(p) = 0 And IsEmpty(AVCalcTab.Range(gRxSvcCopayNR_Arr(p) & vTIERNUM)) = True Then
                gNoteMsg.XSTD = 1
            End If
        Next
    End If
    
    If gNOTE_XRAY_OVRD = 1 Then
        gNoteMsg.XRAY_OVRD = 1
    End If
    
    If gNOTE_OP_OVRD = 1 Then
        gNoteMsg.OP_OVRD = 1
    End If

End Function
```

## Main Function
```
Dim time_start As Double
Dim time_paused As Double
Dim time_end As Double

Option Explicit
Sub StartButton_Click()
  
    'Initialize timekeeping var
    time_start = Timer
    time_paused = 0
    time_end = 0

    initialize_values
    
    MAINLOOP
      
    'output status message
    AVCalcTab.Range("nrSTATUS").value = gSTATUS_MSG
    AVCalcTab.Range("nrNOTESMSG").value = gNoteMsg.SetNoteMsg

    'print time elapsed
    time_end = Timer
    AVCalcTab.Range("nrTIMEKEEP").value = Round(time_end - time_start - time_paused, 4) & " seconds"

    'create image of AV Calculator output
    If gERROR_FLAG = 0 Then Create_Image (gDISABLE_IMAGE)
    
    'clean up variables
    clean_up

End Sub
Sub MAINLOOP()

    Dim AVTier1 As Double
    Dim AVTier2 As Double

    get_plan_level_data
    validate_non_tier_specific_inputs

    Dim i As Integer
    For i = 1 To 2
    
        If gCB_MULTITIER_PLAN = 1 Then
               
            get_cost_sharing_parameters
            validate_tier_specific_inputs
            If gERROR_FLAG = 1 Then Exit Sub
               
            AVTier1 = CalcAV()
            
            get_cost_sharing_parameters ("2")
            validate_tier_specific_inputs ("2")
            If gERROR_FLAG = 1 Then Exit Sub

            AVTier2 = CalcAV("2")
        
            gAV = gTIER1_UTIL * AVTier1 + gTIER2_UTIL * AVTier2

        Else
        
            get_cost_sharing_parameters
            validate_tier_specific_inputs
            If gERROR_FLAG = 1 Then Exit Sub
            
            gAV = CalcAV()

        End If
        
        'Adjust AV value for HSA funds
        If gCB_HSA_IND = 1 Then
            gAV = HSA_ADJUSTMENT(gAV)
        End If
           
        gAV = Application.Min(gAV, 1)
           
        'process implications of final AV
        process_av
    
        'output AV info
        AVCalcTab.Range("nrAVRESULT").value = gAV
        AVCalcTab.Range("nrMETAL_DETERMINATION").value = gMTIER_NAME
        
        If gACTUAL_MTIER_NUM = gDESIRED_MTIER_NUM Or gACTUAL_MTIER_NUM <= 0 Or gCB_CSR_IND = 1 Then
        
            Exit For
            
        ElseIf i = 1 And gACTUAL_MTIER_NUM > 0 Then
        
            time_paused = Timer
            
            Dim rerun As Integer
            rerun = MsgBox("Tier [" & gMTIER_NAME & "] does not match desired metal tier [" & gDESIRED_MTIER_NAME & "]. Do you want to recalculate using the proper metal tier continuance tables?", vbQuestion + vbYesNo)
            
            time_paused = Timer - time_paused

            If rerun = vbYes Then

                gSTATUS_MSG = ""
                gDESIRED_MTIER_NUM = gACTUAL_MTIER_NUM
                gDESIRED_MTIER_NAME = gMTIER_NAME
                HelperTab.Range("nrDESIRED_MTIER").value = gACTUAL_MTIER_NUM
                
            Else
                
                Exit For
            
            End If
        
        End If
    
    Next

End Sub
```